# Geleneksel ML İşleyişi

- Basitçe veri setindeki kategorik değişkenler düzenlenip, Churnler 1614 Churn Olmayanlar 18.375 Olduğu için verisetinde küçük bir filtrelemeye gittik. Burada Churn Olmayanların sayısı Churn olanların 3 katı olarak belirlendi, Ardından Veri seti eğitim ve test olmak üzere 2 ye bölünüp gradient boosting classifier modelinde tahminler yapıldı. Ve gerçek sonuçlar ile karşılaştırıldı.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data = pd.read_excel("../Datasets/newdataFullv3.xlsx")
data = data.dropna()
columns = ['CEK_TUTAR',
 'VADE_GUN',
 'SON5ISTIHBARAT_SONUC',
 'SON5POLITIKA_SONUC',
 'SON5CEK_RENK_ORTALAMASI',
 'SON5KULLANDIRIM',
 'ŞUBE',
 'SIRKET_TURU',
 'MUSTERI_RISK_SEVIYESI',
 'TK_KURUMSAYISISON5',
 'TK_GECIKMEHESAPSON5',
 'TK_GECIKMEBAKIYEORT',
 'TK_NAKDILIMIT',
 'TK_NAKDIRISK',
 'TK_GAYRINAKDILIMIT',
 'BK_KURUMSAYISI',
 'BK_LIMIT',
 'BK_RISK',
 'BK_GECIKMEHESAP',
 'BK_GECIKMEBAKIYE',
 'T2_NAKDILIMIT_TPL',
 'T2_NAKDIRISK_TPL',
 'T2_NAKDIRISK_KV',
 'T2_GAYRINAKDILIMIT',
 'T2_GAYRINAKDIRISK',
 'T2_FKTRNG_LIMIT',
 'T2_FKTRNG_TPLRISK',
 'T2_NAKDIRISK_OV',
 'T2_NAKDIRISK_UV',
 'T2_SORUNLUKOD_RISK',
 'T3_NAKDILIMIT_TPL',
 'T3_NAKDIRISK_TPL',
 'T3_NAKDIRISK_KV',
 'T3_NAKDIRISK_OV',
 'T3_NAKDIRISK_UV',
 'T3_GAYRINAKDILIMIT',
 'T3_GAYRINAKDIRISK',
 'T3_FKTRNG_LIMIT',
 'T3_SORUNLUKOD_RISK',
 'T3_FKTRNG_TPLRISK',
 'T4_NAKDILIMIT_TPL',
 'T4_NAKDIRISK_TPL',
 'T4_NAKDIRISK_KV',
 'T4_NAKDIRISK_OV',
 'T4_NAKDIRISK_UV',
 'T4_GAYRINAKDILIMIT',
 'T4_GAYRINAKDIRISK',
 'T4_FKTRNG_LIMIT',
 'T4_SORUNLUKOD_RISK',
 'T4_FKTRNG_TPLRISK',
 'Churn']

data.head()

,CEK_TUTAR,VADE_GUN,SON5ISTIHBARAT_SONUC,SON5POLITIKA_SONUC,SON5CEK_RENK_ORTALAMASI,SON5KULLANDIRIM,ŞUBE,SIRKET_TURU,MUSTERI_RISK_SEVIYESI,TK_KURUMSAYISISON5,...,T4_NAKDIRISK_TPL,T4_NAKDIRISK_KV,T4_NAKDIRISK_OV,T4_NAKDIRISK_UV,T4_GAYRINAKDILIMIT,T4_GAYRINAKDIRISK,T4_FKTRNG_LIMIT,T4_SORUNLUKOD_RISK,T4_FKTRNG_TPLRISK,Churn
480,17126.636923,86.769231,RED,HAYR,Yesil,KY,IKITELLI,G,0,2,...,2951.2,2148.8,0.0,802.4,0.0,0.0,190000.0,0.0,54430.8,1
680,13715.801250,108.000000,ONAY,EVET,Yesil,KY,MECIDIYEKÖY,G,0,6,...,193811.2,161693.8,0.0,32117.4,44550.0,14400.0,390000.0,0.0,142473.2,1
1967,32722.478958,104.885417,RED,HAYR,Yesil,KY,IZMIT,T,0,5,...,181699.0,0.0,0.0,181699.0,14050.0,11200.0,5690000.0,0.0,496898.0,1
2161,56454.517292,159.562500,RED,HAYR,Siyah,KY,IKITELLI,T,2,14,...,6098286.0,2700782.8,1117431.4,2280071.8,4484608.0,3464409.0,10039565.8,0.0,3619009.6,1
2485,26900.000000,104.200000,RED,HAYR,Yesil,KY,TOPÇULAR,G,0,0,...,0.0,0.0,0.0,0.0,56000.0,0.0,846434.0,9137.0,159051.0,1


- Bu kısımda veri tiplerine göz atarak hangi verileri nasıl kullanacağıma ve kategorik verileri belirleyerek onları makine öğrenmesine uygun hale getirmek için veri tiplerine baktım

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19989 entries, 480 to SC00190565
Data columns (total 51 columns):
CEK_TUTAR                  19989 non-null float64
VADE_GUN                   19989 non-null float64
SON5ISTIHBARAT_SONUC       19989 non-null object
SON5POLITIKA_SONUC         19989 non-null object
SON5CEK_RENK_ORTALAMASI    19989 non-null object
SON5KULLANDIRIM            19989 non-null object
ŞUBE                       19989 non-null object
SIRKET_TURU                19989 non-null object
MUSTERI_RISK_SEVIYESI      19989 non-null int64
TK_KURUMSAYISISON5         19989 non-null int64
TK_GECIKMEHESAPSON5        19989 non-null int64
TK_GECIKMEBAKIYEORT        19989 non-null float64
TK_NAKDILIMIT              19989 non-null float64
TK_NAKDIRISK               19989 non-null float64
TK_GAYRINAKDILIMIT         19989 non-null float64
BK_KURUMSAYISI             19989 non-null int64
BK_LIMIT                   19989 non-null float64
BK_RISK                    19989 non-null float64
BK_

- Bu kısımda kategorik değişkenler numerik hale geliyor. Bunun içinde LabelEncoder adlı kütüphaneyi kullandım

In [4]:
le = preprocessing.LabelEncoder()

objects = ["SON5ISTIHBARAT_SONUC","SON5POLITIKA_SONUC","SON5CEK_RENK_ORTALAMASI","SON5KULLANDIRIM","ŞUBE","SIRKET_TURU"]
for i in objects:  
    data[i] = le.fit_transform(data[i])

- Bu kısımda ise elimde **1614 Churn** ve **18.375 Churn Olmayan** Data olduğu için overfitting yaşamamak adına, bunları ilk önce eşitledim, testlerim sonucunda ise Churn Sayısının 3 Katına kadar Churn olmayan alabileceğimi farkettim ve En optimize değer için bunu 3 kat olarak belirledim

In [5]:
print("Data'nın Bölünmeden Önceki Boyutu", len(data))
dataChurn = data[data.Churn == 1]
dataNormal = data[data.Churn == 0]
data = pd.concat([dataNormal.sample(len(dataChurn)*3, random_state=5,replace=True), dataChurn])
print("Data'nın Yeni Boyutu: ", len(data))

Data'nın Bölünmeden Önceki Boyutu 19989
Data'nın Yeni Boyutu:  6456


- Burada ise veriyi Test ve Eğitim olmak üzere 2'ye böldüm verinin %20 si test %80'i train olarak ayrıldı.

In [6]:
X = data.iloc[:, data.columns != 'Churn']
y = data.Churn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 25)

- Burada çoğu modeli denesemde daha önceki çalışmalarımda ağırlıklı olarak kullandığım ve parametrelerini daha rahat belirleyebildiğim gradient boosting classifier algoritmasını kullandım. Sonucum ve Hata matrisimde aşağıdaki gibi çıktı.

In [7]:
model = GradientBoostingClassifier(learning_rate=0.15,n_estimators=137,subsample=0.7, random_state=51)
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred) 
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(y_test, y_pred))

confusion_matrix:
 [[909  78]
 [141 164]]
accuracy_score:  0.8304953560371517


- En son olarak Majority Vote Algoritmasında kullanmak üzere bunu pickle ile dump ettim. 

In [8]:
filename = 'traditionalml.sav'
pickle.dump(model, open(filename, 'wb'))